In [1]:
import torch
import numpy as np
from torch import nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt

In [2]:
states_in_the_prediction = ['01','02','04','05','06','08','09','10','12','13','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40',
 '41','42','44','45','46','47','48','49','50','51','53','54','55','56']

In [3]:
truth_df = pd.read_csv("../data/CDC/truth-Incident Hospitalizations.csv")
truth_df = truth_df[truth_df['date'] >= '2022-01-01']
truth_df = truth_df[truth_df['location'] != 'US']
truth_df = truth_df[truth_df['location'].isin(states_in_the_prediction)]
truth_df.sort_values(by=['date', 'location'], inplace=True)
unique_dates = truth_df['date'].unique()
unique_states = truth_df['location'].unique()

In [4]:
#### Standardize the data individually for each state
mean_arr = []
std_arr = []
for state in unique_states:
    state_values = truth_df[truth_df['location'] == state]['value'].values
    mean = state_values.mean()
    mean_arr.append(mean)
    std = state_values.std()
    std_arr.append(std)
    truth_df.loc[truth_df['location'] == state, 'norm_value'] = (state_values - mean) / std

In [5]:
weeks = np.zeros([len(unique_dates),len(unique_states)])
for id1,i in enumerate(unique_dates):
    for id2,j in enumerate(unique_states):
        weeks[id1,id2] = truth_df[(truth_df['date']==i) & (truth_df['location']==j)]['value'].values
        # weeks[id1,id2] = truth_df[(truth_df['date']==i) & (truth_df['location']==j)]['norm_value'].values

In [6]:
weeks.shape

(58, 50)

In [7]:
# split the data into train and validation sets
loader_temp = weeks.copy().transpose(1,0)
train_data = loader_temp[:,:-2].copy() #train_data = loader_temp[:,:-4].copy()
train_data = np.concatenate([train_data[:,i:i+10] for i in range(0, 46, 1)], axis = 0)
np.random.shuffle(train_data)
val_data = train_data[:500]
train_data = train_data[500:]
test_data = loader_temp[:,-8:].copy() # test_data = loader_temp[:,-10:].copy()

In [8]:
# convert the numpy arrays to PyTorch tensors
# train_inputs = torch.tensor(train_data[:, :6], dtype=torch.float32)
# train_labels = torch.tensor(train_data[:, 6:], dtype=torch.float32)

# val_inputs = torch.tensor(val_data[:, :6], dtype=torch.float32)
# val_labels = torch.tensor(val_data[:, 6:], dtype=torch.float32)

# test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
# test_labels = torch.tensor(test_data[:, 6:], dtype=torch.float32)

train_inputs = torch.tensor(train_data[:, :6], dtype=torch.float32)
train_labels = torch.tensor(train_data[:, 6:], dtype=torch.float32)

val_inputs = torch.tensor(val_data[:, :6], dtype=torch.float32)
val_labels = torch.tensor(val_data[:, 6:], dtype=torch.float32)

test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_labels = torch.tensor(test_data[:, 6:], dtype=torch.float32)

mean = train_inputs.mean()
std = train_inputs.std()

train_inputs = (train_inputs - mean) / std
val_inputs = (val_inputs - mean) / std
train_labels = (train_labels - mean) / std
val_labels = (val_labels - mean) / std
test_inputs = (test_inputs  - mean) / std

In [9]:
# create the datasets
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
val_dataset =torch.utils.data.TensorDataset(val_inputs, val_labels)

# create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle = False)

In [31]:
# import pickle
# # save
# with open('../data/shuffled_data/MLP_flu_train.pickle', 'wb') as handle:
#     pickle.dump(train_dataset, handle)
# with open('../data/shuffled_data/MLP_flu_val.pickle', 'wb') as handle:
#     pickle.dump(val_dataset, handle)

MLP on residual

In [10]:
class AutoMLP(nn.Module):
    def __init__(self,input_length, output_length,hidden_length):
        super(AutoMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_length, hidden_length),
            nn.ReLU(),
            nn.Linear(hidden_length, hidden_length),
            nn.ReLU(),
            nn.Linear(hidden_length, output_length),
        )
    def forward(self, x):
        return self.model(x)

In [11]:
model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay

In [12]:
best_val = 1e6
for epoch in range(300):
    running_loss = []
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss.append(loss.item())

    
    # validate the model after each epoch
    model.eval()
    with torch.no_grad():
        val_running_loss = []
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss.append(loss.item())
    if epoch % 10 == 0:
        print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
    scheduler.step()
    if np.mean(val_running_loss) < best_val:
        best_val = np.mean(val_running_loss)
        best_model = model

Epoch 1 Training loss: 0.789 Validation loss : 1.689
Epoch 11 Training loss: 0.524 Validation loss : 0.655
Epoch 21 Training loss: 0.477 Validation loss : 0.650
Epoch 31 Training loss: 0.467 Validation loss : 0.674
Epoch 41 Training loss: 0.426 Validation loss : 0.647
Epoch 51 Training loss: 0.390 Validation loss : 0.664
Epoch 61 Training loss: 0.348 Validation loss : 0.592
Epoch 71 Training loss: 0.333 Validation loss : 0.631
Epoch 81 Training loss: 0.286 Validation loss : 0.615
Epoch 91 Training loss: 0.259 Validation loss : 0.635
Epoch 101 Training loss: 0.251 Validation loss : 0.699
Epoch 111 Training loss: 0.217 Validation loss : 0.878
Epoch 121 Training loss: 0.202 Validation loss : 0.742
Epoch 131 Training loss: 0.195 Validation loss : 0.775
Epoch 141 Training loss: 0.181 Validation loss : 0.796
Epoch 151 Training loss: 0.168 Validation loss : 0.957
Epoch 161 Training loss: 0.164 Validation loss : 0.987
Epoch 171 Training loss: 0.157 Validation loss : 0.955
Epoch 181 Training lo

In [20]:
# tryput_1 = (best_model(test_inputs.to(device))* std + mean)
# np.mean(np.abs(np.abs([item for sublist in tryput_1.detach().numpy() for item in sublist]) - np.array(truth_df[truth_df['date'] == '2023-01-28']['value'])))

In [94]:
# pred = best_model(test_inputs.to(device))[:,np.array([False, False, False,True,])]
# result = []
# for i in range(len(pred)):
#     result.append(((pred[i] * std_arr[i])+ mean_arr[i]).tolist())
# flat_list = [item for sublist in result for item in sublist]
# p.mean(np.abs(flat_list - np.array(truth_df[truth_df['date'] == '2023-02-04']['value'])))

In [13]:
test_preds = best_model(test_inputs.to(device)) * std + mean
# test_preds = best_model(test_inputs.to(device)) * std + mean
# print("test error:", torch.mean(torch.abs(test_preds.cpu() - test_labels)))

temp = test_preds[:,np.array([True, False, False,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_1_pred = np.abs(flat_list)

temp = test_preds[:,np.array([False, True, False,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_2_pred = np.abs(flat_list)

temp = test_preds[:,np.array([False, False, True,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_3_pred = np.abs(flat_list)


temp = test_preds[:,np.array([False, False, False,True,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_4_pred = np.abs(flat_list)

In [14]:
groundtruth_0128 = np.array(truth_df[truth_df['date'] == '2023-01-28']['value'])
groundtruth_0204 = np.array(truth_df[truth_df['date'] == '2023-02-04']['value'])

In [15]:
np.stack([groundtruth_0128,groundtruth_0204])[:,0]

array([42, 34])

In [1]:
# load in the updated truth 
new_truth = pd.read_csv("../data/CDC/truth-Incident Hospitalizations 2.csv")
new_truth = new_truth[new_truth['location'] != 'US']
new_truth = new_truth[new_truth['location'].isin(states_in_the_prediction)]
new_truth.sort_values(by=['date', 'location'], inplace=True)
groundtruth_0128 = np.array(new_truth[new_truth['date'] == '2023-01-28']['value'])
groundtruth_0204 = np.array(new_truth[new_truth['date'] == '2023-02-04']['value'])
groundtruth_0211 = np.array(new_truth[new_truth['date'] == '2023-02-11']['value'])
groundtruth_0218 = np.array(new_truth[new_truth['date'] == '2023-02-18']['value'])

NameError: name 'pd' is not defined

In [16]:
print('1 week ahead: ',np.mean(np.abs(week_1_pred - groundtruth_0128)))
print('2 week ahead: ',np.mean(np.abs(week_2_pred - groundtruth_0204)))
print('3 week ahead: ',np.mean(np.abs(week_3_pred - groundtruth_0128)))
print('4 week ahead: ',np.mean(np.abs(week_4_pred - groundtruth_0204)))

1 week ahead:  21.5026469039917
2 week ahead:  22.64774673461914
3 week ahead:  29.41070137023926
4 week ahead:  43.41663303375244


In [52]:
GLEAM_01_23_pd = pd.read_csv("../data/GLEAM/2023-01-23-MOBS-GLEAM_FLUH.csv")
GLEAM_01_28_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '1 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
GLEAM_02_04_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '2 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
print('GLEAM 1 week ahead: ',np.mean(np.abs(GLEAM_01_28_pred - groundtruth_0128)))
print('GLEAM 2 week ahead: ',np.mean(np.abs(GLEAM_02_04_pred - groundtruth_0204)))

GLEAM 1 week ahead:  18.984057145383236
GLEAM 2 week ahead:  15.82940462261393


In [91]:
def plot():
    x = np.arange(1,5)
    for i in range(50):
        plt.plot(np.arange(1,3), np.abs(test_preds[i,:].detach().numpy())[:2],label="Prediction", marker = "*")
        plt.plot(np.arange(1,3), np.stack([GLEAM_01_28_pred,GLEAM_02_04_pred])[:,i],label="GLEAM Prediction", marker = "v")
        plt.plot(np.arange(1,3), np.abs(test_preds[i,:].detach().numpy())[2:],label="Prediction Shifted", marker = "*")
        # plt.plot(x, week_4_pred[i],label="Truth Used for Prediction", marker = ".")
        plt.plot(np.arange(1,3),np.stack([groundtruth_0128,groundtruth_0204])[:,i],label="Truth", marker = "o")
        plt.title(states_in_the_prediction[i])
        plt.xlabel("Num of Weeks Ahead")
        plt.ylabel("Number of Flu Cases")
        plt.legend(loc='best')
        # plt.fill_between(x, pred_reshaped[i,:,0] + ground_truth[i,:],pred_reshaped[i,:,2] + ground_truth[i,:],color = 'green',alpha = 0.2)
        #plt.fill_between(x, pred_reshaped[i,:,0]  + ground_truth[i,:],pred_reshaped[i,:,2]  + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.show()

In [144]:
# proportions = [.90, .10]
# lengths = [int(p * len(train_loader)) for p in proportions]
# lengths[-1] = len(train_loader) - sum(lengths[:-1])
# tr_dataset, vl_dataset = torch.utils.data.random_split(train_loader, lengths)

In [ ]:
# class AutoMLP(nn.Module):
#     def __init__(self,input_length, hidden_length, output_length):
#         super(AutoMLP, self).__init__()
#         self.input_length = input_length
#         self.hidden_length = hidden_length
#         self.output_length = output_length
#         self.fc1 = nn.Linear(self.input_length, self.hidden_length)
#         self.fc2 = nn.Linear(self.hidden_length, self.output_length)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)

In [ ]:
def get_plots_quantile_prediction(npz_file_location):
    npz = np.load(npz_file_location)
    pred = npz['prediction']
    
    # truth = npz['truth']
    ground_truth = np.resize(dfg_date_filtered_truth_filtered[dfg_date_filtered_truth_filtered['date'] == '2023-01-21']['value'].to_numpy(), (4,50)).T

    # print(pred.shape,truth.shape)
    # if "quantile_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4 ,3)
    #     truth_reshaped = truth.reshape(50, 4)
    # elif "dropout_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4)
    #     truth_reshaped = truth.reshape(50, 4)
    pred_reshaped = pred.reshape(50, 4 ,3)
    pred_actual_ground_truth = np.resize(dfg_date_filtered_truth_filtered[dfg_date_filtered_truth_filtered['date'].isin(pd.date_range('2023-01-21', '2023-01-28', freq='W-SAT'))]['value'].to_numpy(), (4,50)).T
    # truth_reshaped = truth.reshape(50, 4)

    x = np.arange(1,5)
    # states = list(states_data[1])
    states = dfg_date_filtered_truth_filtered['location_name'].unique()
    # states = states_in_the_prediction
    for i in range(50):
        plt.plot(x, pred_reshaped[i,:,1] + ground_truth[i,:],label="Prediction", marker = "*")
        plt.plot(x, pred_reshaped[i,:,0]  + ground_truth[i,:],label="Lower Bound", marker = "v")
        plt.plot(x, pred_reshaped[i,:,2]  + ground_truth[i,:],label="Upper Bound", marker = "^")
        plt.plot(x, ground_truth[i,:],label="Truth Used for Prediction", marker = ".")
        plt.plot(np.arange(1,3), pred_actual_ground_truth[i,:2],label="Truth", marker = "o")
        plt.title(states[i])
        plt.xlabel("Num of Weeks Ahead")
        plt.ylabel("Number of Flu Cases")
        plt.legend(loc='best')
        # plt.fill_between(x, pred_reshaped[i,:,0] + ground_truth[i,:],pred_reshaped[i,:,2] + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.fill_between(x, pred_reshaped[i,:,0]  + ground_truth[i,:],pred_reshaped[i,:,2]  + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.show()